<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/geemap_supervised1_modis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=p8aEVvAeGEkgfCgy9bKBuncfIug2zk06paxuvuzA3UU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhdsr1Dmw3mq7fwUM81-6-XHluT2hp75Y4uzCVrL_KRh6l4Sxl1-Z4

Successfully saved authorization token.


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
wdpa_polygon = ee.FeatureCollection('WCMC/WDPA/current/polygons')
# select the pa in cambodia
aoi = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Ang Trapeng Thmor'))


NameError: ignored

In [ ]:


#image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]')
image=ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]'))
#landsat=image.clip(aoi)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

map4 = geemap.Map(center=[12.5657, 105],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(image, vis_params, 'Landsat-8')

map4.addLayerControl()
map4

Map(center=[12.5657, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [ ]:
# Use MODIS land cover, IGBP classification, for training.
#modis = ee.Image('MODIS/051/MCD12Q1/2011_01_01').select('Land_Cover_Type_1')
modis=ee.Image('MODIS/051/MCD12Q1/2019_02_01').select('Land_Cover_Type_2')


In [ ]:
# Sample the input imagery to get a FeatureCollection of training data.
#training = image.addBands(modis).sample(numPixels=5000, seed=0,geometries=True)
training=image.addBands(modis).sample(numPixels=5000,seed=0,geometries=True)

In [ ]:
# Make a Random Forest classifier and train it.
#classifier = ee.Classifier.smileRandomForest(10).train(training, 'Land_Cover_Type_1')
classifier=ee.Classifier.smileRandomForest(10).train(training,'Land_Cover_Type_2')

In [ ]:
# Classify the input imagery.
#classified = image.classify(classifier)
classified=image.classify(classifier)

Test accuracy


In [ ]:
# Get a confusion matrix representing resubstitution accuracy.
#trainAccuracy = classifier.confusionMatrix()
trainAccuracy=classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

EEException: ignored

In [ ]:
# Sample the input with a different random seed to get validation data.
validation = image.addBands(modis)\
                  .sample(numPixels=5000,seed=1)\
                  .filter(ee.Filter.neq('B1', None)) # Filter the result to get rid of any null pixels.


In [ ]:
# Classify the validation data.
validated = validation.classify(classifier)

In [ ]:
# Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('Land_Cover_Type_2', 'classification')
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

Validation error matrix:  [[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 240, 0, 0, 0, 0, 0, 66, 1, 0, 0, 6, 1, 98], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 60, 0, 0, 0, 0, 0, 601, 0, 16, 0, 10, 0, 328], [0, 0, 3, 0, 0, 0, 0, 0, 36, 0, 0, 1, 3, 0, 50], [0, 0, 4, 0, 0, 0, 0, 0, 62, 0, 6, 0, 13, 0, 93], [0, 0, 3, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 0, 4], [2, 0, 0, 0, 0, 0, 0, 2, 31, 2, 8, 0, 87, 1, 493], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 27], [0, 0, 46, 0, 0, 1, 0, 0, 259, 0, 17, 2, 199, 0, 2081]]
Validation overall accuracy:  0.6042834267413931


In [ ]:
classification = image.classify(classifier).multiply(100).clip(aoi)

In [ ]:
# Define a palette for the IGBP classification.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

In [ ]:
Map = geemap.Map(center=[12,105], zoom= 8)
Map.addLayer(classification.randomVisualizer())
Map

Map(center=[12, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…